In [4]:
# Section 1
import numpy as np
import matplotlib.pyplot as plt
import h5py
import copy

# Section 2
from scipy import signal
import scipy as sp

# Sections 4-5
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
# from sklearn.neural_network import MLPClassifier # use Tensorflow
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay

# 1. Loading the data #

Load and preprocess the data: copied from notebook 4 of exercises from week 10

In [6]:
# data_path = ''
fs = 24400 # Hz

# Subjects
n_subjects = 2
subjs_info_loading = {'Subject2' : {'Name' : 'p2-t2', 'Vars' : ['Baseline', 'Angio36', 'RR15', 'TV500']},
                      'Subject21' : {'Name' : 'p6-t6', 'Vars' : ['Baseline', 'Angio36', 'RR15', 'TV125']}
                     }
subjs_info_final = {'Subject2' : {'Name' : 'p2-t2', 'Vars' : ['Baseline', 'Angio36', 'RRC', 'TVC']},
                    'Subject21' : {'Name' : 'p6-t6', 'Vars' : ['Baseline', 'Angio36', 'RRC', 'TVC']}
                   }
animals_labels = ['p2-t2', 'p6-t6']

def load_data_all_subjects(subjs_info_loading, subjs_info_final, fs, type_data = 'Field_Data_Neuro'):
    subjects_names = list(subjs_info_loading.keys())
    subjs_info = subjs_info_final
    all_data = {}
    print('Start loading ... \n')
    for subject in subjects_names:
        name_subj, data_struct = load_data_one_subject(subject, subjs_info_loading, fs, type_data = type_data)
        all_data[name_subj] = data_struct
        print('================== %s loaded. =================='%subject)
    return all_data


def load_data_one_subject(subject, subjs_info_loading, fs, type_data = 'Field_Data_Neuro'):
    name_subj_to_stock = subjs_info_loading[subject]['Name']
    vars_to_load = subjs_info_loading[subject]['Vars']
    data_struct = {}
    
    # file = h5py.File('data\\' + subject + '.mat','r')
    file = h5py.File(subject + '.mat','r')
    
    # To get the names of the fields after decoding ASCII
    all_field_names = get_field_names(file)

    for var in vars_to_load:
        id_field = np.where(all_field_names == var)[0]
        curr_reference_data1 = file['Vagus_Data_Stimuli'][type_data][id_field][0][0]
        curr_reference_data2 = file[curr_reference_data1][0][0]
        final_data = np.transpose(np.asarray(file[curr_reference_data2]))
        
        if var == 'TV800' or var == 'TV500' or var == 'TV125': 
            var_name = 'TVC'
        elif var == 'RR15' or var == 'RR20': 
            var_name = 'RRC'
        else:
            var_name = var
            
        data_struct[var_name] = {}
        data_struct[var_name]['Data'] = final_data
        n_time_pts = np.shape(final_data)[-1]
        data_struct[var_name]['Time_pts'] = np.linspace(0, n_time_pts/fs, n_time_pts)
        
    return name_subj_to_stock, data_struct

def get_field_names(file):
    n_fields,_ = np.shape(file['Vagus_Data_Stimuli']['stimuli_name'])
    all_field_names = []
    for field in range(n_fields):
        curr_reference_field = file['Vagus_Data_Stimuli']['stimuli_name'][field][0]
        curr_field_ASCII = file[curr_reference_field]
        curr_field = decode_ASCII(curr_field_ASCII)
        all_field_names.append(curr_field)
    return np.asarray(all_field_names)


def decode_ASCII(numbers_array):
    name = ''
    squeezed_numbers = np.squeeze(numbers_array)
    for n in squeezed_numbers:
        name += chr(n)
    return name

data = load_data_all_subjects(subjs_info_loading, subjs_info_final, fs)
print('Loading completed \n')

#print('Showing the data file:')
#for key1 in data.keys():
#    print('=========== %s ==========='%key1)
#    for key2 in data[key1].keys():
#        print('--- %s'%key2)
#        for key3 in data[key1][key2].keys():
#            print(' - %s'%key3)
#            print('Shape : ', np.shape(data[key1][key2][key3]))

def cut_all_data_to_established_duration_per_challenge(data):
    '''
    This function is a wrap-up to the function 'cut_data_one_pig_to_established_duration_per_challenge'.
    '''
    
    new_data_struct = {}
    for pig in data.keys():
        print('================== Working on Pig %s =================='%pig)
        data_curr_pig = data[pig]
        data_curr_pig_cut = cut_data_one_pig_to_established_duration_per_challenge(data_curr_pig)
        new_data_struct[pig] = data_curr_pig_cut
        
    return new_data_struct


def cut_data_one_pig_to_established_duration_per_challenge(data_one_pig):
    ''' 
    This function is used to cut the data for each challenge to the duration shown in Suppl. Table 1 in Vallone et al., 2021. 
    We take the first part of the data for each challenge (arbitrary choice). 
    '''
    new_struct = copy.deepcopy(data_one_pig)
    
    dur_baseline = 5 #min
    dur_RRC = 2 #min
    dur_TVC = 2 #min
    
    for challenge in new_struct.keys():
        data_curr_chal = new_struct[challenge]['Data']
        time_pts_curr_chal = new_struct[challenge]['Time_pts']
        t_end_sec = time_pts_curr_chal[-1]
        
        if challenge == 'Baseline': t_end_sec = dur_baseline * 60
        elif challenge == 'RRC': t_end_sec = dur_RRC * 60
        elif challenge == 'TVC': t_end_sec = dur_TVC * 60
            
#         print('Challenge %s , t_end_sec %0.3f'%(challenge, t_end_sec))
            
        id_t_end_curr_chal = find_specific_time_index(time_pts_curr_chal, t_end_sec)
        new_struct[challenge]['Data'] = data_curr_chal[:,:id_t_end_curr_chal]
        new_struct[challenge]['Time_pts'] = time_pts_curr_chal[:id_t_end_curr_chal]
    
    return new_struct

def find_specific_time_index(time_pts, t):
    t_id = np.argmin(np.abs(time_pts - t))
    return t_id

print('Start cutting data ... \n')
cut_data = cut_all_data_to_established_duration_per_challenge(data)
print('Data cutting completed \n')

Start loading ... 

================== Subject2 loaded. ==================
================== Subject21 loaded. ==================
Loading completed 

Start cutting data ... 

================== Working on Pig p2-t2 ==================
================== Working on Pig p6-t6 ==================
Data cutting completed 



Data is structured pig -> challenge -> data, time

In [27]:
print('Showing the data files:')
for key1 in cut_data.keys():
    print('=========== %s ==========='%key1)
    for key2 in cut_data[key1].keys():
        print('--- %s'%key2)
        for key3 in cut_data[key1][key2].keys():
            print(' - %s'%key3)
            print('Shape : ', np.shape(cut_data[key1][key2][key3]), ', type: ', type(cut_data[key1][key2][key3]))
            

Showing the data files:
=========== p2-t2 ===========
--- Baseline
 - Data
Shape :  (8, 7319999) , type:  <class 'numpy.ndarray'>
 - Time_pts
Shape :  (7319999,) , type:  <class 'numpy.ndarray'>
--- Angio36
 - Data
Shape :  (8, 14780415) , type:  <class 'numpy.ndarray'>
 - Time_pts
Shape :  (14780415,) , type:  <class 'numpy.ndarray'>
--- RRC
 - Data
Shape :  (8, 2927999) , type:  <class 'numpy.ndarray'>
 - Time_pts
Shape :  (2927999,) , type:  <class 'numpy.ndarray'>
--- TVC
 - Data
Shape :  (8, 2927999) , type:  <class 'numpy.ndarray'>
 - Time_pts
Shape :  (2927999,) , type:  <class 'numpy.ndarray'>
=========== p6-t6 ===========
--- Baseline
 - Data
Shape :  (16, 7319999) , type:  <class 'numpy.ndarray'>
 - Time_pts
Shape :  (7319999,) , type:  <class 'numpy.ndarray'>
--- Angio36
 - Data
Shape :  (16, 14940159) , type:  <class 'numpy.ndarray'>
 - Time_pts
Shape :  (14940159,) , type:  <class 'numpy.ndarray'>
--- RRC
 - Data
Shape :  (16, 2927999) , type:  <class 'numpy.ndarray'>
 - T

# 2. Preprocessing #

The paper says they use a 4-order Butterworth filter [1000, 6000] Hz. 

Note that we have fs = 24400 Hz, hence by Shannon the recorded signal can have no components above 2*fs 48000. 

Why do we bandpass at 6k Hz then???

We highpass at 1kHz to remove any forms of noise, notably due to the organism itself. 

In [9]:
f_low = 1000 # [Hz]
f_high = 6000 # [Hz]
N = 4 # order of the filter
sos = signal.butter(N, [f_low, f_high], 'bandpass', analog=False, fs=fs, output='sos')

In [10]:
def bandpass_data(data, sos):
    print('Bandpassing the data... \n')
    new_data = copy.deepcopy(data)
    for pig in data.keys():
        print('=========== %s ==========='%pig)
        for challenge in data[pig].keys():
            new_data[pig][challenge] = signal.sosfilt(sos, cut_data[pig][challenge]['Data'])
    return new_data        

Note that here we remove the 'time' field, which is not used

In [11]:
# takes ~40sec to run
data_filtered = bandpass_data(cut_data, sos)

Bandpassing the data... 

=========== p2-t2 ===========
=========== p6-t6 ===========


Sub-sampling of the signal by a factor 2: 

(Why???)

In [12]:
# takes ~20sec to run
data_subsampled = copy.deepcopy(data_filtered)
# data_subsampled = data_blank
for pig in data.keys():
    for challenge in data[pig].keys():
        data_subsampled[pig][challenge] = data_subsampled[pig][challenge][:,0:-1:2]

# 3. Split into test/train, perform windowing, extract features #

In [ ]:
data_final = data_subsampled
kf = KFold(10)
for pig in data_final.keys():
	for challenge, values in enumerate(data_final[pig]):
		for train_index, test_index in kf.split(values.T):


In [ ]:
classifiers = [RandomForestClassifier]
for classifier in classifiers:
	kf = KFold(10)
	for train_index, test_index in kf.split(data_final):
		gridseearch = GridSearchCV()


Following the paper, we use a 90-10 train-test split

In [13]:
def split_data(data, n_folds):
	return_data = np.zeros()
	kf = KFold(n_folds)
	for train_index, test_index in kf.split(data):
		pass


IndentationError: expected an indented block (Temp/ipykernel_24992/109244714.py, line 5)

In [14]:
def split_data(data, n_folds):
    data_train = {'Baseline': {}, 'Angio36' : {}, 'RRC': {}, 'TVC': {}}
    data_test = dict.fromkeys(data.keys(), {})
    for challenge in data.keys():
        L = data[challenge].shape[1]
        data_train[challenge]= data[challenge][:,0:int(tt_split*L)]
        data_test[challenge]= data[challenge][:,int(tt_split*L):]
    return data_train, data_test

In [15]:
# Could make a dict of dicts but for two pigs it's unecessary
tt_split = 0.9
pig1 = 'p2-t2'
pig2 = 'p6-t6'
pig1_train_data, pig1_test_data = split_data(data_subsampled[pig1], tt_split)
pig2_train_data, pig2_test_data = split_data(data_subsampled[pig2], tt_split)

Following this we have the following data structure: four separate sets, each set contains the four challenges, for each challenge the data

Compute features, as given in the paper. 

Note the structure used:

- compute_features computes features on the time series for a single channel, for a single challenge
- windowing_and_features generates features for a given challenge; it iterates in time to create windows, inside this loop it iterates on each channel and concatenates the result to make a feature vector [(num_features*num_channels)]; 
- data_windowing_and_features creates the features array and the labels vector by combining the features on the four challenges

Other implementation detail: it's easier to perform windowing and features computation in the same loop, because that way the dimension you gain from windowing is lost in computing features, then you don't need to use 3-d arrays

In [16]:
def compute_features(data):
    '''
    :param data: vector [timesteps]
    :return features: vector of features for that time series [9]
    '''
    mean = np.mean(data)
    variance = np.var(data)
    skew = sp.stats.skew(data)
    kurtosis = sp.stats.kurtosis(data)
    std = np.std(data)
    mav = np.mean(np.absolute(data))
    max = np.amax(data)
    amp = 0 #
    wl = 0  #
    pow = np.sqrt(np.mean(data ** 2))

    for i in range(len(data) - 1):
        if abs(data[i + 1] - data[i]) >= std:
            amp += 1
        wl += abs(data[i + 1] - data[i])
    
    features = np.array([mean, variance, skew, kurtosis, mav, max, amp, wl, pow])
    return features

def windowing_and_features(data, windowSize_n, windowOverlap_n):
    '''
    :param data: ndarray [channels x timesteps]
    :return: ndarray [windows x (features*channels)]
    '''
    windowStep_n = windowSize_n - windowOverlap_n
    num_windows = int((data.shape[1]-windowSize_n)/windowStep_n)
    num_features = 9
    num_channels = data.shape[0]
    features = np.empty((num_windows, num_features*num_channels))
    for i in range(num_windows): # iterate over windows
        for j in range(num_channels): # iterate over channels
            window = data[j, i*windowStep_n:i*windowStep_n+windowSize_n]
            features_channel = compute_features(window)
            features[i,j*num_features:(j+1)*num_features] = features_channel
    return features

def data_windowing_and_features(data, windowSize_n, windowOverlap_n):
    '''
    :param data: dictionnary with four keys (challenges), each challenge contains an ndarray [channels x timesteps]
    :return features: ndarray [(channels*windows) x (features*channels)]
    :return labels: vector [(channels*windows)]
    '''
    num_features = 9
    num_channels = data[list(data.keys())[0]].shape[0] # height of the first dict element
    features = np.empty((0,num_features*num_channels))
    labels = np.empty((0))
    for challenge in data.keys():
        challenge_features = windowing_and_features(data[challenge], windowSize_n, windowOverlap_n)
        label = list(data).index(challenge) # converts to an index
        challenge_labels = label* np.ones(challenge_features.shape[0])
        features = np.append(features, challenge_features, axis = 0)
        labels = np.append(labels, challenge_labels)
    return features, labels

In [17]:
# [TODO] to find the best ws, several sizes were tested from ws = 50ms to ws = 1s with steps of 50 ms.

windowSize_t = 50 # [ms]
windowSize_n = int(windowSize_t * fs * 0.001) #[timesteps]
windowOverlap_train_t = 20 #[ms]
windowOverlap_train_n = int(windowOverlap_train_t * fs * 0.001)
windowOverlap_test_n = 0

print('Generating training data for pig 1...')
pig1_train_X, pig1_train_y = data_windowing_and_features(pig1_train_data, windowSize_n, windowOverlap_train_n)
print('Finished generating training data for pig 1')
print('Generating testing data for pig 1...')
pig1_test_X, pig1_test_y = data_windowing_and_features(pig1_test_data, windowSize_n, windowOverlap_test_n)
print('Finished generating testing data for pig 1')

print('Generating training data for pig 2...')
pig2_train_X, pig2_train_y = data_windowing_and_features(pig2_train_data, windowSize_n, windowOverlap_train_n)
print('Finished generating training data for pig 2')
print('Generating testing data for pig 2...')
pig2_test_X, pig2_test_y = data_windowing_and_features(pig2_test_data, windowSize_n, windowOverlap_test_n)
print('Finished generating testing data for pig 2')

Generating training data for pig 1...
Finished generating training data for pig 1
Generating testing data for pig 1...
Finished generating testing data for pig 1
Generating training data for pig 2...
Finished generating training data for pig 2
Generating testing data for pig 2...
Finished generating testing data for pig 2


The vectors look fine at a first glance ... (note that pig2 has twice as many recording channels so the features vector is two times longer)

In [19]:
print(pig1_train_X.shape)
print(pig1_train_y.shape)
print(pig1_test_X.shape)
print(pig1_test_y.shape)
print(pig2_train_X.shape)
print(pig2_train_y.shape)
print(pig2_test_X.shape)
print(pig2_test_y.shape)

(17178, 72)
(17178,)
(1141, 72)
(1141,)
(17276, 144)
(17276,)
(1148, 144)
(1148,)


In [20]:
print(pig1_train_X)

[[ 2.06182545e-09  1.01400968e-11 -7.28075476e-02 ...  3.09000000e+02
   2.21830473e-03  2.49513404e-06]
 [-5.62823297e-09  1.02704590e-11 -3.41194766e-02 ...  3.22000000e+02
   2.23127705e-03  2.47020535e-06]
 [ 6.50087471e-10  1.04629099e-11 -8.80026964e-03 ...  3.40000000e+02
   2.19553394e-03  2.41157685e-06]
 ...
 [-2.76207252e-09  1.21895792e-11 -1.24975353e-01 ...  3.33000000e+02
   2.27811433e-03  2.54226255e-06]
 [ 3.58589470e-10  1.13805201e-11  5.57459530e-02 ...  3.53000000e+02
   2.25277658e-03  2.42579245e-06]
 [ 2.39236873e-09  1.23116876e-11  1.07057651e-01 ...  3.24000000e+02
   2.19928159e-03  2.40340710e-06]]


In [21]:
print(pig1_test_y)

[0. 0. 0. ... 3. 3. 3.]


# 4. RF Classifier #

hyperparameters to vary for each window size:

"During training, a grid-search was performed in which the number of trees varied in [50, 100, 200, 400], while their max- imal depth in [5, 10, 20]"

Make the previous section a function so we can test different window sizes:

In [22]:
def generate_tt(data, windowSize_t, fs):
    '''
    :param data: dict of arrays, ie data for one pig
    '''
    tt_split = 0.9 # magic number from paper
    train_data, test_data = split_data(data, tt_split)

    windowSize_n = int(windowSize_t * fs * 0.001) #[timesteps]
    windowOverlap_train_t = 20 #[ms] # magic number form paper
    windowOverlap_train_n = int(windowOverlap_train_t * fs * 0.001)
    windowOverlap_test_n = 0 # magic number form paper
    train_X, train_y = data_windowing_and_features(train_data, windowSize_n, windowOverlap_train_n)
    test_X, test_y = data_windowing_and_features(test_data, windowSize_n, windowOverlap_test_n)
    return train_X, train_y, test_X, test_y

In [ ]:
pig1 = 'p2-t2'
pig2 = 'p6-t6'

for windowSize_t in np.arange(50, 1000, 50): # [ms]
    print('Window size: %.i ms'%windowSize_t)
    pig1_train_X, pig1_train_y, pig1_test_X, pig1_test_y = generate_tt(data_subsampled[pig1], windowSize_t, fs)
    pig2_train_X, pig2_train_y, pig2_test_X, pig2_test_y = generate_tt(data_subsampled[pig2], windowSize_t, fs)
    # [TODO]: fill in the ML
    class_weights = {}
    clf = RandomForestClassifier(n_estimators=100, max_depth=None)
    clf.fit(pig1_train_X, pig1_train_y)
    pig1_predicted_y = clf.predict(pig1_test_X)

    clf.fit(pig2_train_X, pig2_train_y)
    pig2_predicted_y = clf.predict(pig2_test_X)
    
    

# 5. Other Classifiers # 

MLP

hyperparameters to vary for each window size:

"for multi layer perceptron (MLP), the three architectures [128, 128, 128, 128], [128, 512, 512, 128], and [128, 256, 512, 512] were tested, where the numbers indicate the number of hidden neurons from the first to the fourth layer. For MLP, training was performed with a learning rate of 1 × 10−3 for 30 epochs and a batch size of 128"

In [ ]:
for windowSize_t in np.arange(50, 1000, 50): # [ms]
    print('Window size: %.i ms'%windowSize_t)
    pig1_train_X, pig1_train_y, pig1_test_X, pig1_test_y = generate_tt(data_subsampled[pig1], windowSize_t, fs)
    pig2_train_X, pig2_train_y, pig2_test_X, pig2_test_y = generate_tt(data_subsampled[pig2], windowSize_t, fs)
    # [TODO]: fill in the ML
    # Should use tensorflow

SVM

hyperparameters to vary for each window size:

"for SVM, the kernel function was selected between the radial basis function (rbf) function or the sigmoid
function and the L2 regularization value varied in [1 × 10−2 , 1 × 10−1 , 1], with 1 indicating no regularization"

In [ ]:
BAC_pig1 = np.empty(0)
BAC_pig2 = np.empty(0)
for windowSize_t in np.arange(50, 1000, 50): # [ms]
    print('Window size: %.i ms'%windowSize_t)
    pig1_train_X, pig1_train_y, pig1_test_X, pig1_test_y = generate_tt(data_subsampled[pig1], windowSize_t, fs)
    # Adapted from: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
    clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
    clf.fit(pig1_train_X, pig1_train_y)
    pig1_predicted_y = clf.predict(pig1_test_X)
    BAC1 = balanced_accuracy_score(pig1_test_y, pig1_predicted_y)
    BAC_pig1.append(BAC1)
    
    pig2_train_X, pig2_train_y, pig2_test_X, pig2_test_y = generate_tt(data_subsampled[pig2], windowSize_t, fs)
    clf.fit(pig2_train_X, pig2_train_y)
    pig2_predicted_y = clf.predict(pig2_test_X)
    BAC2 = balanced_accuracy_score(pig2_test_y, pig2_predicted_y)
    BAC_pig2.append(BAC2)

plt.plot(np.arange(50, 1000, 50), BAC_pig1)
plt.plot(np.arange(50, 1000, 50), BAC_pig2)
plt.xlabel('Time window length [ms]')
plt.ylabel('Balanced Accuracy [%]')
plt.legend('p2-t2', 'p6-t6')
plt.show()

windowSize_t = 500 #[ms]
pig1_train_X, pig1_train_y, pig1_test_X, pig1_test_y = generate_tt(data_subsampled[pig1], windowSize_t, fs)
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(pig1_train_X, pig1_train_y)
# The forum post below gives code to generate confusion matrices exactly like in the paper:
# https://stackoverflow.com/questions/40264763/how-can-i-make-my-confusion-matrix-plot-only-1-decimal-in-python
ConfusionMatrixDisplay.from_estimator(clf, pig1_test_X, pig1_test_y)
plt.show()

pig2_train_X, pig2_train_y, pig2_test_X, pig2_test_y = generate_tt(data_subsampled[pig2], windowSize_t, fs)
clf.fit(pig2_train_X, pig2_train_y)
ConfusionMatrixDisplay.from_estimator(clf, pig2_test_X, pig2_test_y)
plt.show()

K-Means

hyperparameters to vary for each window size:

"for KNN, the number of neighbors varied in [3, 5, 7, 9, 11, 13]"

In [ ]:
for windowSize_t in np.arange(50, 1000, 50): # [ms]
    print('Window size: %.i ms'%windowSize_t)
    pig1_train_X, pig1_train_y, pig1_test_X, pig1_test_y = generate_tt(data_subsampled[pig1], windowSize_t, fs)
    pig2_train_X, pig2_train_y, pig2_test_X, pig2_test_y = generate_tt(data_subsampled[pig2], windowSize_t, fs)
    # [TODO]: fill in the ML